# 태스크 2: 질문에 답변하는 데 Amazon Bedrock 사용

이 노트북에서는 Bedrock Nova Lite 모델을 사용해 쿼리에 대한 정보 응답을 제공하는 방법을 학습합니다. 이때 전체 관련 컨텍스트와 함께 모델에 요청을 보내고, 그에 대한 응답을 받는 방식으로 작동하며, 사전에 문서를 준비하거나 인덱싱할 필요 없이 사실에 기반한 답변을 얻는 과제를 해결합니다.

이 노트북에서는 **검색 증강 생성(RAG)** 의 역할을 시뮬레이션하지만, 실제로 RAG를 사용하지는 않습니다. 이러한 접근 방식은 짧은 문서 또는 단독 애플리케이션에 효과적입니다. 모델에 전송된 프롬프트에 대규모 엔터프라이즈 문서가 모두 들어가지 않을 경우, 엔터프라이즈 수준의 질문에 대한 답변으로 규모가 조정되지 않을 수 있습니다.

**질문 답변(QA)** 은 자연어로 제기되는 사실적인 쿼리에 대한 답변을 추출하는 것이 포함된 중요한 태스크입니다. 일반적으로 QA 시스템은 정형 또는 비정형 데이터가 포함된 지식 기반에 대해 쿼리를 처리하고, 정확한 정보로 응답을 생성합니다. 높은 정확도를 보장하는 것은 특히 엔터프라이즈 사용 사례를 위한 유용하고 안정적이고 신뢰할 수 있는 질문 답변 시스템을 개발하는 데 있어서 핵심적인 요인입니다.


## 시나리오

AnyCompany에서 제조하는 특정 차량 모델의 타이어 교체에 대한 정보를 제공하기 위해 질문에 답변하는 모델을 테스트하는 상황을 시뮬레이션해 봅니다. 먼저 ‘제로샷’ 접근 방식으로 모델을 쿼리하여 훈련 데이터만을 기반으로 관련 답변을 제공할 수 있는지 확인합니다.

그런데 가짜 차량 모델을 시험해 보고 비슷한 응답을 받았을 때 드러났듯이, 이 모델은 좀 더 일반적인 답변에서 ‘할루시네이션’ 현상이 일어나는 것 같다는 걸 깨닫습니다. 이는 각 모델의 타이어에 대한 세부 사항을 제공하는 Example Company의 실제 차량 설명서로 모델의 훈련을 보강할 필요가 있다는 것을 암시합니다.

이 실습에서는 외부 데이터 없이 이러한 ‘검색 증강 생성’(RAG) 접근 방식을 시뮬레이션합니다. AnyCompany 모델 Z 차량의 타이어 교체 방법을 설명하는 자세한 설명서 발췌를 제공합니다. 이제 모델이 상황에 맞는 예시 콘텐츠를 활용하여 사용자 지정된 정확한 답변을 제공할 수 있는지 테스트합니다.

## 태스크 2.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [2]:
# Create a service client by name using the default session.
import json
import os
import boto3

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

# Configure the modelId and request/response format
modelId = "amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"

# Imports to display markdown outputs
from IPython.display import display_markdown, Markdown, clear_output

In [3]:
"""
Nova Lite Adapter:

This code block contains helper functions for using Nova Lite.
"""

def format_for_nova_lite(prompt_text):
    """Format the prompt for Nova Lite's expected message structure."""
    return {
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt_text}]
            }
        ],
        "inferenceConfig": {
            "maxTokens": 2048,
            "temperature": 0,
            "topP": 0.9
        }
    }

def parse_nova_lite_response(response_body):
    """Parse the response from Nova Lite."""
    if 'output' in response_body and 'message' in response_body['output']:
        message = response_body['output']['message']
        if 'content' in message and isinstance(message['content'], list):
            # Extract text from each content item
            texts = []
            for content_item in message['content']:
                if isinstance(content_item, dict) and 'text' in content_item:
                    texts.append(content_item['text'])
            return ' '.join(texts)
    
    # Fallback if the response format is different
    return str(response_body)

## 태스크 2.2: 모델 지식을 사용한 Q&A
이 섹션에서는 Bedrock 서비스에서 제공한 모델을 사용하여 훈련 단계 동안 획득한 지식을 토대로 질문에 답변하려고 합니다.

이 태스크에서는 Amazon Bedrock 클라이언트의 invoke_model() 메서드를 사용합니다. 이 메서드를 사용하는 데 필요한 필수 파라미터는 Amazon Bedrock 모델 ARN을 나타내는 modelId와 태스크에 대한 프롬프트인 body입니다.

body 프롬프트는 선택하는 파운데이션 모델 공급자에 따라 변경됩니다. 아래 내용을 자세히 살펴봅니다.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Bedrock 서비스에서 제공한 모델을 사용하여 훈련 단계 동안 획득한 지식을 토대로 질문에 답변하려고 합니다.

In [4]:
prompt_data = """You are an helpful automotive assistant. Answer questions in a concise way. When answering questions about car maintenance or repairs, provide a detailed, numbered list of steps. Assume all cars, including the AnyCompany AC8, have a spare tire. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** 이어지는 태스크에서 코드를 실행할 때 *ThrottlingException* 메시지가 표시되고 코드 실행이 재시도될 수도 있습니다. 여기서 실행하는 코드에는 지수 백오프를 통해 실패한 요청을 자동으로 재시도하는 안정적인 오류 처리 기능이 포함되어 있습니다. 서비스 할당량 사용 시 오류 메시지가 표시되는 것은 정상적인 현상입니다. 이 오류 메시지를 통해 실제 사용 가능한 상태의 애플리케이션에서 API 제한 처리 방식을 확인할 수 있습니다.

## 태스크 2.3: JSON 본문을 전달하여 모델을 간접 호출하고 응답 생성

In [5]:
try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
    
    # Read from the response stream
    response_body = json.loads(response.get('body').read())

    # Parse the output
    outputText = parse_nova_lite_response(response_body)

    # Display the output text in markdown format
    display_markdown(Markdown(outputText))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

To fix a flat tire on your AnyCompany AC8, follow these steps:

1. **Ensure Safety**:
   - Park the car on a flat, stable surface.
   - Engage the parking brake.
   - Turn on hazard lights.

2. **Gather Tools**:
   - Jack
   - Lug wrench
   - Spare tire
   - Wheel chocks (if available)
   - Tire repair kit (optional, for temporary fixes)

3. **Loosen Lug Nuts**:
   - Use the lug wrench to loosen the lug nuts slightly while the car is still on the ground. Do not remove them yet.

4. **Position the Jack**:
   - Locate the jacking points on your AnyCompany AC8 (refer to the owner's manual if unsure).
   - Place the jack under the jacking point and raise the car until the flat tire is about 6 inches off the ground.

5. **Remove Lug Nuts and Tire**:
   - Fully remove the lug nuts and set them aside.
   - Carefully remove the flat tire and set it aside.

6. **Mount the Spare Tire**:
   - Align the spare tire with the lug nut posts.
   - Push the spare tire onto the posts until it sits flush against the hub.

7. **Tighten Lug Nuts**:
   - Hand-tighten the lug nuts in a star pattern to ensure even tightening.
   - Use the lug wrench to fully tighten the lug nuts.

8. **Lower the Car**:
   - Carefully lower the car using the jack until the spare tire touches the ground.
   - Fully lower the car and then remove the jack.

9. **Final Tightening**:
   - Once the car is on the ground, give the lug nuts a final tightening in the same star pattern.

10. **Check Tire Pressure**:
    - Check the pressure of the spare tire and ensure it is safe to drive.

11. **Drive Carefully**:
    - Spare tires are not meant for long distances or high speeds. Drive cautiously to the nearest repair shop.

Optional: If you have a tire repair kit and the flat is repairable, you can patch the tire temporarily and drive to a repair shop.


이 모델은 바람이 빠진 자동차 타이어를 교체하는 프로세스를 설명하는 답변을 제공하지만, 동일한 설명이 모든 자동차에 유효할 수 있습니다. 안타깝게도 예비 타이어가 없는 AnyCompany AC8에는 올바른 답변이 아닙니다. 모델이 자동차의 타이어 교체에 대한 지침이 포함된 데이터로 훈련되었기 때문입니다.

가상의 자동차 브랜드와 모델인 Amazon Tirana에 대해 동일한 질문을 제시하면 이 문제의 또 다른 예를 확인할 수 있습니다.

In [6]:
# Usage
prompt_data = """How can I fix a flat tire on my Amazon Tirana?
Answer: Here are the steps to fix a flat tire:
"""

try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
    
    # Read from the response stream
    response_body = json.loads(response.get('body').read())

    # Parse the output
    outputText = parse_nova_lite_response(response_body)

    # Display the output text in markdown format
    display_markdown(Markdown(outputText))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Fixing a flat tire on your Amazon Tirana (assuming it's a bicycle or a similar vehicle) involves several steps. Here’s a detailed guide to help you through the process:

### Tools and Materials Needed:
- Tire levers
- Patch kit or tire repair plug (if needed)
- Pump or CO2 inflator
- Wrench (if applicable)
- A clean, flat surface to work on

### Steps to Fix a Flat Tire:

1. **Safety First:**
   - Ensure the bike is stable and won't tip over. Use a bike stand if available.
   - Make sure the area is well-lit and free of debris.

2. **Remove the Wheel:**
   - If the bike has quick-release levers, open them to release the wheel.
   - For bikes with nuts, use a wrench to loosen the nuts holding the wheel in place.
   - Carefully remove the wheel from the bike.

3. **Remove the Tire from the Wheel:**
   - Use tire levers to pry one side of the tire away from the wheel rim. Start at the valve stem to make it easier.
   - Once one side is free, you can use your hands to pull the tire off the wheel completely.

4. **Remove the Inner Tube:**
   - Locate the valve stem and pull the inner tube out of the tire. Note how the tire was seated on the rim to make reinstallation easier.
   - Inspect the inner tube for punctures. If you find one, proceed to the next step.

5. **Repair the Inner Tube:**
   - **Patch Kit:** Clean the area around the puncture with a cloth. Apply rubber cement to both the patch and the tube, then press the patch onto the tube and let it sit for a few minutes.
   - **Tire Plug:** Insert the plug into the puncture hole and use the tool provided to press it in firmly.

6. **Re-inflate the Inner Tube (Optional):**
   - Before reinstalling the inner tube, you can partially inflate it to check for leaks. This makes it easier to seat the tube in the tire.

7. **Reinstall the Inner Tube:**
   - Place the inner tube inside the tire, ensuring the valve stem aligns with the valve hole on the rim.
   - Make sure the tube isn’t twisted and is seated evenly around the tire.

8. **Reinstall the Tire:**
   - Start by seating one side of the tire back onto the rim.
   - Use your hands to work the rest of the tire back onto the rim. If it’s difficult, use tire levers to help, but be careful not to pinch the inner tube.
   - Ensure the tire is seated evenly around the rim.

9. **Reinstall the Wheel:**
   - Place the wheel back into the bike’s dropouts.
   - If you have quick-release levers, close them. If you have nuts, tighten them by hand first, then use a wrench to snug them up in a star pattern.

10. **Inflate the Tire:**
    - Use a pump or CO2 inflator to inflate the tire to the recommended pressure, usually found on the sidewall of the tire.

11. **Final Check:**
    - Spin the wheel to ensure it rotates freely and check for any signs of the tire coming off the rim.
    - Make any necessary adjustments to the quick-release or nuts.

### Tips:
- Always carry a patch kit and a pump or CO2 inflator when riding.
- Regularly check your tires for wear and potential punctures.
- Consider using tubeless tires for added durability and puncture resistance.

By following these steps, you should be able to fix a flat tire on your Amazon Tirana and get back on the road quickly.

프롬프트 질문이 주어지면 모델에서 현실적인 답변을 제공할 수 없습니다.

이 문제를 해결하고 모델에서 차량 모델에 유효한 특정 지침을 기반으로 답변을 제공하도록 하려면 프롬프트의 일부로 추가 지식 기반을 제공하여 모델의 지식을 즉시 보강할 수 있습니다.

이 방법을 사용하여 애플리케이션을 개선하는 방법을 살펴보겠습니다.

다음은 AnyCompany AC8 설명서에서 발췌한 것입니다(사실 실제 설명서는 아니지만 그렇게 취급해 주십시오). 또한 이 문서는 Nova Lite 컨텍스트 창 전체를 담을 수 있을 만큼 짧아서 편리합니다.

```plain
Tires and Tire Pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to Find Recommended Tire Pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the Tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for Using the Tire Mobility Kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```

In [7]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### 이제 전체 발췌문을 질문과 함께 모델에 전달합니다.

In [8]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide in 5 steps with conclusion.
#
{context}
#

Question: {question}
Answer:"""

### 태스크 2.4: boto3를 통해 모델을 간접 호출하여 응답 생성

In [9]:
try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
    
    # Read from the response stream
    response_body = json.loads(response.get('body').read())

    # Parse the output
    outputText = parse_nova_lite_response(response_body)

    # Display the output text in markdown format
    display_markdown(Markdown(outputText))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Based on the information provided, here is a step-by-step guide to fix a flat tire on your AnyCompany AC8:

### Step 1: Locate the Tire Mobility Kit
- **Action**: Open the tailgate or trunk of your vehicle.
- **Explanation**: The tire mobility kit is stored under the lining of the luggage area.

### Step 2: Access the Tire Mobility Kit
- **Action**: Lift up the lining of the luggage area to access the tire mobility kit.
- **Explanation**: This will give you access to the necessary tools and materials to temporarily seal the puncture.

### Step 3: Seal the Puncture
- **Action**: Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
- **Explanation**: The kit will include a sealant and a pump. The instructions will guide you on how to apply the sealant to the tire and reinflate it.

### Step 4: Secure the Kit
- **Action**: After using the kit, make sure to securely put it back in its original location.
- **Explanation**: Proper storage of the kit ensures it is ready for future use and keeps your vehicle's interior organized.

### Step 5: Seek Professional Assistance
- **Action**: Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.
- **Explanation**: The tire mobility kit is a temporary solution. It is important to have the tire professionally inspected and repaired or replaced as soon as possible.

### Conclusion
By following these steps, you can temporarily fix a flat tire on your AnyCompany AC8 using the tire mobility kit. However, it is crucial to seek professional help to ensure the tire is properly repaired or replaced to maintain the safety and performance of your vehicle.

모델에서 컨텍스트를 이해하고 관련 답변을 생성하는 데 시간이 걸리기 때문에 응답을 몇 초 동안 기다려야 하므로 사용자 경험이 저하될 수 있습니다.

또한 Bedrock은 모델에서 토큰을 생성할 때 서비스에서 출력을 생성하는 스트리밍 기능도 지원합니다. 다음은 이를 구현하는 방법의 예입니다.

In [10]:
try:
    # Create the body from the prompt
    body = format_for_nova_lite(prompt_data)

    # Invoke the model
    response = bedrock_client.invoke_model_with_response_stream(
        body=json.dumps(body), 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )

    stream = response.get('body')
    output = []
    
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            # Extract text from Nova Lite's response format
            if 'contentBlockDelta' in chunk_obj and 'delta' in chunk_obj['contentBlockDelta']:
                text = chunk_obj['contentBlockDelta']['delta'].get('text', '')
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Based on the information provided, here is a step-by-step guide to fixing a flat tire on your AnyCompany AC8:

### Step 1: Locate the Tire Mobility Kit
1. **Open the tailgate or trunk** of your vehicle.
2. **Lift up the lining of the luggage area** to access the tire mobility kit.

### Step 2: Seal the Puncture
1. **Follow the instructions provided with the tire mobility kit** to seal the puncture in the tire. This typically involves inserting the sealant into the tire and using the kit's tools to seal the puncture.

### Step 3: Reinflate the Tire
1. **Check the recommended tire pressure** for your vehicle. This can be found on the inflation label on the driver's side B-pillar or in your vehicle's manual.
2. **Follow the instructions provided on the air pump** and inflate the tire(s) to the correct pressure.

### Step 4: Calibrate the Tire Pressure
1. **In the center display of your vehicle, open the "Car status" app**.
2. **Navigate to the "Tire pressure" tab**.
3. **Press the "Calibrate pressure" option and confirm the action**.
4. **Drive the car for a few minutes at a speed above 30 km/h** to calibrate the tire pressure.

### Step 5: Seek Professional Assistance
1. **Contact AnyCompany or an appropriate service** for assistance with disposing of and replacing the used sealant bottle.
2. **Have the punctured tire replaced or repaired by a professional** as soon as possible.

### Conclusion
By following these steps, you can temporarily fix a flat tire using the tire mobility kit and then seek professional help to ensure your vehicle is safe and properly maintained. Remember, the tire mobility kit is only a temporary solution and should not be used for extended periods or high speeds.

이 응답은 타이어 교체 방법에 대한 요약된 단계별 지침을 제공합니다. 

지금까지 검색 증강 생성(RAG) 또는 증강 프로세스를 활용하여 제공된 특정 컨텍스트와 정보에 알맞게 큐레이팅된 응답을 생성할 수 있는 방법을 알아보았습니다.

### 직접 해보기
- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 개선된 출력을 얻으십시오.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **결론**을 계속 진행합니다.